Install the Monstera package from the repo.

In [2]:
import Pkg
# activate a temporary environment
Pkg.activate(mktempdir())
Pkg.add([
    Pkg.PackageSpec(name="Monstera", url="https://github.com/eascarrunz/Monstera.jl"),
    Pkg.PackageSpec(name="HTTP"),
])
using Monstera, HTTP;

  Activating new project at `/run/user/1000/jl_FYsPtx`
    Updating git-repo `https://github.com/eascarrunz/Monstera.jl`
   Resolving package versions...
    Updating `/run/user/1000/jl_FYsPtx/Project.toml`
  [cd3eb016] + HTTP v1.10.5
  [a7c66a61] + Monstera v1.0.0-DEV `https://github.com/eascarrunz/Monstera.jl#main`
    Updating `/run/user/1000/jl_FYsPtx/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
  [d1d4a3ce] + BitFlags v0.1.8
  [944b1d66] + CodecZlib v0.7.4
  [34da2185] + Compat v4.14.0
  [f0e56b4a] + ConcurrentUtilities v2.4.1
  [864edb3b] + DataStructures v0.18.18
  [460bff9d] + ExceptionUnwrapping v0.1.10
  [cd3eb016] + HTTP v1.10.5
  [692b3bcd] + JLLWrappers v1.5.0
  [682c06a0] + JSON v0.21.4
  [e6f89c97] + LoggingExtras v1.0.3
  [739be429] + MbedTLS v1.1.9
  [a7c66a61] + Monstera v1.0.0-DEV `https://github.com/eascarrunz/Monstera.jl#main`
  [4d8831e6] + OpenSSL v1.4.2
  [bac558e1] + OrderedCollections v1.6.3
  [69de0a69] + Parsers v2.8.1
  [aea7be01] + PrecompileTools v

Initialise a taxon set. Node labels will be mapped to the names in the taxon set to assign taxon IDs to the nodes.

In [3]:
taxonset = TaxonSet();

Download an MCMC sample of 1000 Newick trees with 567 taxa.

In [4]:
r = HTTP.get("https://raw.githubusercontent.com/eascarrunz/Monstera.jl/main/data/hashrftrees/567-taxa-1000-trees.tre");
io = IOBuffer(r.body);

Parse the Newick trees and store them as `RoundaboutTree` objects in a vector. Passing `taxonset` automatically populates the taxon set based on the outer node labels and maps the nodes to numeric taxon IDs.

In [5]:
trees = Newick.read(io, Vector{RoundaboutTree}, taxonset);

Call the `distance` function once for precompilation.

In [6]:
@time distance(trees[1:2], :ci);

  3.788873 seconds (5.34 M allocations: 365.658 MiB, 99.80% compilation time)


Compute the upper triangle of the clustering information distance matrix from the 100 trees. This takes about 1.4 seconds to run on a laptop with a Ryzen 7 6800HS CPU.

The distances are currently computed as half the distances computed by `TreeDist`.

In [13]:
@time distance(trees[1:100], :ci)

  1.416917 seconds (1.19 M allocations: 256.535 MiB, 1.47% gc time)


100×100 Matrix{Float64}:
 0.0  2.75633  4.4255   5.17554  …  4.8111   5.34417  5.6317   5.55706
 0.0  0.0      4.49923  4.06133     4.71057  4.77044  4.91982  4.93409
 0.0  0.0      0.0      4.89703     4.66354  5.17844  4.01575  3.96897
 0.0  0.0      0.0      0.0         4.94567  4.63162  5.29632  4.90222
 0.0  0.0      0.0      0.0         4.80733  4.59184  5.17149  4.86192
 0.0  0.0      0.0      0.0      …  4.54137  4.56828  4.91127  4.88225
 0.0  0.0      0.0      0.0         4.11695  4.15291  4.52128  4.45199
 0.0  0.0      0.0      0.0         4.55311  4.70814  5.57567  5.68436
 0.0  0.0      0.0      0.0         5.33933  5.12093  4.81288  5.39069
 0.0  0.0      0.0      0.0         5.32373  5.11687  5.0614   5.49002
 ⋮                               ⋱                             
 0.0  0.0      0.0      0.0         3.5857   3.81251  5.04693  5.05114
 0.0  0.0      0.0      0.0         3.88784  3.97894  5.01491  5.33617
 0.0  0.0      0.0      0.0         4.06022  4.00418  5.202

That matrix was computed based only on the biparitions that are different between each pair of trees. If we use all the bipartitions, it takes almost 1.5 minutes.

In [8]:
@time distance(trees[1:100], :ci; diffonly=false)

 86.368022 seconds (1.21 M allocations: 11.985 GiB, 0.21% gc time, 0.05% compilation time)


100×100 Matrix{Float64}:
 0.0  2.75633  4.4255   5.17554  …  4.8111   5.34417  5.6317   5.55706
 0.0  0.0      4.49923  4.06133     4.71057  4.77044  4.91982  4.93409
 0.0  0.0      0.0      4.89703     4.66354  5.17844  4.01575  3.96897
 0.0  0.0      0.0      0.0         4.94567  4.63162  5.29632  4.90222
 0.0  0.0      0.0      0.0         4.80733  4.59184  5.17149  4.86192
 0.0  0.0      0.0      0.0      …  4.54137  4.56828  4.91127  4.88225
 0.0  0.0      0.0      0.0         4.11695  4.15291  4.52128  4.45199
 0.0  0.0      0.0      0.0         4.55311  4.70814  5.57567  5.68436
 0.0  0.0      0.0      0.0         5.33933  5.12093  4.81288  5.39069
 0.0  0.0      0.0      0.0         5.32373  5.11687  5.0614   5.49002
 ⋮                               ⋱                             
 0.0  0.0      0.0      0.0         3.5857   3.81251  5.04693  5.05114
 0.0  0.0      0.0      0.0         3.88784  3.97894  5.01491  5.33617
 0.0  0.0      0.0      0.0         4.06022  4.00418  5.202

The full 1000 tree matrix can be computed in about 4 minutes.

In [9]:
@time distance(trees)

231.245260 seconds (26.65 M allocations: 34.345 GiB, 0.37% gc time, 0.00% compilation time)


1000×1000 Matrix{Float64}:
 0.0  2.75633  4.4255   5.17554  …  5.7205   5.72457  5.00139  5.02603
 0.0  0.0      4.49923  4.06133     5.93423  6.02717  4.25524  5.78556
 0.0  0.0      0.0      4.89703     5.65512  5.6913   5.41252  6.00769
 0.0  0.0      0.0      0.0         5.88089  5.53748  4.78058  5.27086
 0.0  0.0      0.0      0.0         5.33503  5.33298  4.86751  6.05797
 0.0  0.0      0.0      0.0      …  5.104    4.72003  5.48905  6.38525
 0.0  0.0      0.0      0.0         4.98434  4.51051  5.15777  6.71335
 0.0  0.0      0.0      0.0         6.02888  6.21276  5.30559  5.46525
 0.0  0.0      0.0      0.0         5.94724  6.03167  5.07     5.57064
 0.0  0.0      0.0      0.0         6.05521  6.31313  5.22726  5.81831
 ⋮                               ⋱                             
 0.0  0.0      0.0      0.0         4.63276  4.52476  4.23652  5.00938
 0.0  0.0      0.0      0.0         4.77842  4.88525  4.6677   4.50058
 0.0  0.0      0.0      0.0         4.86225  4.96468  4.6

The `distance` function also has a Robinson-Foulds mode. The matrix with 1000 trees takes about 2.6 seconds.

In [10]:
distance(trees[1:2], :rf)     # precompilation	
@time distance(trees, :rf)

  2.580908 seconds (14.17 M allocations: 4.711 GiB, 4.84% gc time)


1000×1000 Matrix{Int64}:
 0  66  94  106  110  138  134  112  …  192  172  190  178  186  174  194
 0   0  86   86   96  126  122  116     178  172  182  182  190  158  190
 0   0   0   94  108  128  124  116     174  184  186  180  188  168  198
 0   0   0    0   76  106   98  118     184  174  186  186  184  170  184
 0   0   0    0    0   96   96  124     180  178  186  174  176  172  194
 0   0   0    0    0    0   16  112  …  176  186  200  170  172  168  194
 0   0   0    0    0    0    0  110     176  180  192  164  164  164  196
 0   0   0    0    0    0    0    0     188  192  196  180  186  174  192
 0   0   0    0    0    0    0    0     178  182  180  180  186  162  182
 0   0   0    0    0    0    0    0     184  192  190  188  198  180  192
 ⋮                      ⋮            ⋱              ⋮                 
 0   0   0    0    0    0    0    0      56  102  116  114  110  100  124
 0   0   0    0    0    0    0    0      30   98  116  110  114  102  118
 0   0   0    0 